### Importing Libraries

In [1]:
import random
import import_ipynb
import json
import torch
from neural_net import NeuralNet  #importing ipynb neural_net
from nlp_preprocess import bag_of_words, tokenize   #importing ipynb nlp_preprocess

importing Jupyter notebook from neural_net.ipynb
importing Jupyter notebook from nlp_preprocess.ipynb


##### Reading the intents 

In [2]:
with open('intents.json', 'r') as json_data:
    intents = json.load(json_data)

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

### Load the weight trained model and Assign values to variables

In [4]:
FILE = "modeltrained.pth"
data = torch.load(FILE)

input_size = data["input_size"]
hidden_size = data["hidden_size"]
output_size = data["output_size"]
vocabulary = data['vocabulary']
tags = data['tags']
model_state = data["model_state"]


print("Architecture:")
model = NeuralNet(input_size, hidden_size, output_size).to(device)
model.load_state_dict(model_state)
model.eval()

Architecture:


NeuralNet(
  (l1): Linear(in_features=83, out_features=8, bias=True)
  (l2): Linear(in_features=8, out_features=8, bias=True)
  (l3): Linear(in_features=8, out_features=8, bias=True)
  (relu): ReLU()
)

In [ ]:
bot_name = "Therapize"

#Looping for the entire conversation with the user
print("Hey, Welcome to Therapize. Let's chat! (type 'quit' to exit)")
print("-"*117)
while True:
    #Try entering something like a 'hello' for starter; quits out of loop with exit word set as "quit"
    sentence = input("You: ")
    if sentence == "quit":
        break
    #all the live sentence going through the same preprocess; storing results in output
    sentence = tokenize(sentence)
   # print("\nTokenized:",sentence,"\n") if needed to print the live tokenized form of input
    X = bag_of_words(sentence, vocabulary)
   # print("Bag of words:",X,"\n")       if needed to see the bag of words vector for the input
    X = X.reshape(1, X.shape[0]) 
    X = torch.from_numpy(X).to(device)
    output = model(X)   
    _,predicted = torch.max(output, dim=1)   #predicted label having the highest value

    tag = tags[predicted.item()]  #getting the index from item, to later check if tag exists in intents for response

    probs = torch.softmax(output, dim=1)   #applying softmax layer for probability scores at the o/p layer
    prob = probs[0][predicted.item()]
    if prob.item() > 0.75:           #setting the prob as confidence, stricter the value more hard margin for match
        for intent in intents['intents']:
            if tag == intent["tag"]:
                print(f"{bot_name}: {random.choice(intent['responses'])}")  #generating random choice on any response from intents
                print("-"*117)

    else:
        print(f"{bot_name}: Sorry, we didn't understand that")
        print("-"*117)


Hey, Welcome to Therapize. Let's chat! (type 'quit' to exit)
---------------------------------------------------------------------------------------------------------------------
You: Hey 
Therapize: Hi there, how can I help? :)
---------------------------------------------------------------------------------------------------------------------
You: How are you doing
Therapize: Hi there, how can I help? :)
---------------------------------------------------------------------------------------------------------------------
You: What is anxiety
Therapize: Everyone has feelings of anxiety at some point in their life. For example, you may feel worried and anxious about sitting an exam, or having a medical test or job interview. During times like these, being anxious is completely normal, But for some people it's difficult to come out of that state, it takes them weaks or months to feeling relaxed again. They lose control over such worries and it starts to affect their daily lives, that's w